In [1]:
!kaggle datasets download -d quangvinhhuynh/marketing-and-retail-analyst-e-comerce

Dataset URL: https://www.kaggle.com/datasets/quangvinhhuynh/marketing-and-retail-analyst-e-comerce
License(s): apache-2.0




  0%|          | 0.00/48.0M [00:00<?, ?B/s]
  2%|▏         | 1.00M/48.0M [00:00<00:15, 3.25MB/s]
  6%|▋         | 3.00M/48.0M [00:00<00:06, 6.97MB/s]
 10%|█         | 5.00M/48.0M [00:00<00:05, 8.84MB/s]
 15%|█▍        | 7.00M/48.0M [00:00<00:04, 9.90MB/s]
 19%|█▉        | 9.00M/48.0M [00:01<00:03, 10.3MB/s]
 23%|██▎       | 11.0M/48.0M [00:01<00:03, 10.1MB/s]
 27%|██▋       | 13.0M/48.0M [00:01<00:03, 10.3MB/s]
 31%|███▏      | 15.0M/48.0M [00:01<00:03, 10.7MB/s]
 35%|███▌      | 17.0M/48.0M [00:01<00:03, 10.8MB/s]
 40%|███▉      | 19.0M/48.0M [00:02<00:02, 11.0MB/s]
 44%|████▍     | 21.0M/48.0M [00:02<00:02, 11.0MB/s]
 48%|████▊     | 23.0M/48.0M [00:02<00:02, 11.3MB/s]
 52%|█████▏    | 25.0M/48.0M [00:02<00:02, 11.4MB/s]
 56%|█████▋    | 27.0M/48.0M [00:02<00:01, 11.2MB/s]
 60%|██████    | 29.0M/48.0M [00:02<00:01, 11.4MB/s]
 65%|██████▍   | 31.0M/48.0M [00:03<00:01, 11.4MB/s]
 69%|██████▉   | 33.0M/48.0M [00:03<00:01, 11.5MB/s]
 73%|███████▎  | 35.0M/48.0M [00:03<00:01, 11.6MB/s]
 

In [2]:
import shutil

filename = 'marketing-and-retail-analyst-e-comerce.zip' # 압축 해제할 파일 이름
extrac_dir = '데이터셋' # 압축 해제할 폴더 이름
archive_format = 'zip' # 압축 파일 형태

shutil.unpack_archive(filename, extrac_dir, archive_format)

In [3]:
import os

os.listdir('데이터셋/')

['capstone_data_cleaned.csv',
 'customers.csv',
 'orders.csv',
 'order_items.csv',
 'payments.csv',
 'products.csv',
 'Retail_dataset.xlsx']